## Preprocessing on Credit Card Spending

### Imports

In [1]:
# necessary imports
import pandas as pd

# surpress warnings
pd.options.mode.chained_assignment = None  # default='warn'

### Preprocessing

To quickly reiterate the preprocesssing plan laid out in `eda.ipynb`:

Preprocessing for individual DataFrames:

* `boa_df`:
    * Drop null records as they are of `Type` "Payment" and I'm only interested in "Sale"
    * Convert `Amount` to positive values
* `chase_df`:
    * Drop null records, same reasoning as above
    * Convert `Amount` to positive values
* `apple_df`:
    * Change record with `Merchant` "Best Buy" with `Category` "Other" to "Shopping"
    * Perform mapping for `Category` column:
        * | BoA/Chase | Apple |
            | --- | --- |
            | Food & Drink | Restaurants |
            | Groceries | Grocery |

General preprocessing:
* Rename all column names to the corresponding name desired in the table schema:
    * | column_name | boa_df | chase_df | apple_df |
        | --- | --- | --- | --- |
        | transaction_date | Posted Date | Transaction Date | Transaction Date |
        | merchant | Payee | Description | Merchant |
        | category | Category | Category | Category |
        | amount | Amount | Amount | Amount (USD) |
        | card | "BoA" | "Chase" | "Apple" |
* Drop all other columns that are not enumerated in the schema
* For consistency, convert to lower case all values in the `Merchant` column. Additionally get rid of special characters `*` and replace with the empty character `''`

Reading in the datasets:

In [2]:
# Read in the datasets
boa_csv = "data/bofa_spending_mod.csv"
boa_df = pd.read_csv(boa_csv)

chase_csv = "data/chase_spending.csv"
chase_df = pd.read_csv(chase_csv)

apple_csv = "data/apple_spending.csv"
apple_df = pd.read_csv(apple_csv)

#### Preprocessing Helper Functions

Below, I define a some general helper functions for preprocessing. Mostly simple filters and transformations. Honestly, these modifications aren't difficult and can be done in one line, but I decided to abstract them away into a helper function.

For the preprocessing needed for `apple_df`, I will perform it in `preprocess_apple()`, as it is specific to just one record.

In [3]:
# Takes in a DataFrame `df` and removes records with `Type` payment
# Returns the new filtered DataFrame
def remove_payment(df):
    return df[df['Type']=='Sale']

# Takes in a DataFrame `df` and converts `amount` values to positive
# Returns the new filtered DataFrame
def convert_amount(df):
    df['amount'] = abs(df['amount'])
    return df

# Takes in a DataFrame `df` and transforms the values in `column` (containing Strings) into lowercase
# Returns the new transformed DataFrame
def to_lower(df, column):
    df[column] = df[column].str.lower()
    return df

# Takes in a DataFrame `df` and uses `mapping` to map (rename) column names to
# those desired in the table schema
# Returns the DataFrame with the renamed columns
def transform_columns(df, mapping):
    return df.rename(columns=mapping)

# Takes in a DataFrame `df` and adds a column `card` containing values `card_name`
# for every record in the DataFrame
# Returns the new DataFrame with the added column
def add_card_column(df, card_name):
    df['card'] = [card_name]*len(df) # array containing len(df) number of card_names
    return df

def remove_special(df, column, pat, repl):
    df[column] = df[column].str.replace(pat, repl, regex=True)
    return df

#### Bank of America Preprocessing

Below, I perform preprocessing on `boa_df`:

In [4]:
schema_columns = ['transaction_date', 'merchant', 'category', 'amount', 'card']

# Function that performs all the preprocessing functions for the Bank of America DataFrame
# Returns the cleaned DataFrame
def preprocess_boa():
    # Filter for `Type` "Sale"
    clean_boa_df = remove_payment(boa_df)

    # Create boa-specific column mapping to table schema to rename columns
    boa_mapping = {'Posted Date':'transaction_date', 'Payee':'merchant',
                   'Category':'category', 'Amount':'amount'}
    clean_boa_df = transform_columns(clean_boa_df, boa_mapping)

    # Convert `Amount` to positive values
    clean_boa_df = convert_amount(clean_boa_df)

    # Add `card` column containing 'BoA' for each record
    clean_boa_df = add_card_column(clean_boa_df, 'BoA')

    # Remove special characters specified
    clean_boa_df = remove_special(clean_boa_df, column='merchant', pat='[*]', repl='')

    # Lowercase values in the `merchant` column
    clean_boa_df = to_lower(clean_boa_df, 'merchant')

    # Keep desired columns as specified in table schema
    clean_boa_df = clean_boa_df[schema_columns]

    # Return the cleaned DataFrame
    return clean_boa_df

Call `preprocess_boa()` and check if it was properly cleaned:

In [5]:
clean_boa_df = preprocess_boa()
clean_boa_df.head(10)

,transaction_date,merchant,category,amount,card
0,05/09/2023,galpao gaucho cupertino,Food & Drink,175.44,BoA
2,04/28/2023,tst teaspoon - saratoga,Food & Drink,5.50,BoA
3,04/25/2023,chipotle 1031,Food & Drink,24.53,BoA
4,04/24/2023,chipotle 1031,Food & Drink,13.84,BoA
5,04/24/2023,dino's restaurant,Food & Drink,38.00,BoA
6,04/24/2023,ckeikes place palo alto,Food & Drink,36.33,BoA
7,04/24/2023,ikea east palo alto,Shopping,7.81,BoA
8,04/24/2023,ikea east palo alto,Shopping,313.13,BoA
9,04/24/2023,dinos grill,Food & Drink,33.03,BoA
10,04/24/2023,safeway #1224,Groceries,11.98,BoA


It looks like the cleaning was successful. Values in `merchant` have been converted to all lowercase and special characters have been replaced with `''`. All values in `amount` have been converted to positive values and an additional `card` column was added to the DataFrame. All desired columns have been renamed in accordance to the names specified in the table schema.

Now, to repeat this process $2$ more times for `chase_df` and `apple_df`.

#### Chase Preprocessing

Performing preprocessing on `chase_df`:

In [6]:
# Function that performs all the preprocessing functions for the Chase DataFrame
# Returns the cleaned DataFrame
def preprocess_chase():
    # Filter for `Type` "Sale"
    clean_chase_df = remove_payment(chase_df)

    # Create boa-specific column mapping to table schema to rename columns
    chase_mapping = {'Transaction Date':'transaction_date', 'Description':'merchant',
                     'Category':'category', 'Amount':'amount'}
    clean_chase_df = transform_columns(clean_chase_df, chase_mapping)

    # Convert `Amount` to positive values
    clean_chase_df = convert_amount(clean_chase_df)

    # Add `card` column containing 'Chase' for each record
    clean_chase_df = add_card_column(clean_chase_df, 'Chase')

    # Remove special characters specified
    clean_chase_df = remove_special(clean_chase_df, column='merchant', pat='[*]', repl='')

    # Lowercase values in the `merchant` column
    clean_chase_df = to_lower(clean_chase_df, 'merchant')

    # Keep desired columns as specified in table schema
    clean_chase_df = clean_chase_df[schema_columns]

    # Return the cleaned DataFrame
    return clean_chase_df

Call `preprocess_chase()` on `chase_df` and check if it was properly cleaned:

In [7]:
clean_chase_df = preprocess_chase()
clean_chase_df.head(10)

,transaction_date,merchant,category,amount,card
0,05/25/2023,kindle svcso67rm7ps3,Shopping,9.99,Chase
2,05/12/2023,netflix,Bills & Utilities,19.99,Chase
3,05/11/2023,spotify usa,Bills & Utilities,4.99,Chase
5,05/03/2023,paypal steam games,Entertainment,29.22,Chase
6,05/02/2023,paypal steam games,Entertainment,19.99,Chase
7,04/30/2023,izumi sushi,Food & Drink,154.95,Chase
8,04/30/2023,berkeley bowl west,Groceries,5.99,Chase
9,04/29/2023,paypal gwdconcepts,Shopping,10.48,Chase
11,04/14/2023,paypal steam games,Entertainment,29.99,Chase
12,04/12/2023,netflix.com,Bills & Utilities,19.99,Chase


#### Apple Preprocessing

Performing preprocessing on `apple_df`:

In [8]:
# Function that performs all the preprocessing functions for the Apple DataFrame
# Returns the cleaned DataFrame
def preprocess_apple():
    # Create boa-specific column mapping to table schema to rename columns
    apple_mapping = {'Transaction Date':'transaction_date', 'Merchant':'merchant',
                     'Category':'category', 'Amount (USD)':'amount'}
    clean_apple_df = transform_columns(apple_df, apple_mapping)

    # Add `card` column containing 'Apple' for each record
    clean_apple_df = add_card_column(clean_apple_df, 'Apple')

    # Remove special characters specified
    clean_apple_df = remove_special(clean_apple_df, column='merchant', pat='[*]', repl='')

    # Lowercase values in the `merchant` column
    clean_apple_df = to_lower(clean_apple_df, 'merchant')

    # Keep desired columns as specified in table schema
    clean_apple_df = clean_apple_df[schema_columns]

    # Convert that one record with 'Best Buy' as `Merchant` to `Category` 'Shopping'
    clean_apple_df.loc[clean_apple_df['merchant']=='Best Buy', 'category'] = 'Shopping'

    # Perform Apple `Category` mapping to that of BoA/Chase
    # Realized that for entries without mappings, null values were returned. To circumvent this
    # identity mappings were created for 'Shopping' and 'Other'
    apple_category_mapping = {'Shopping':'Shopping', 'Restaurants':'Food & Drink',
                              'Grocery':'Groceries', 'Other':'Other'}
    clean_apple_df['category'] = clean_apple_df['category'].map(apple_category_mapping)

    # Return the cleaned DataFrame
    return clean_apple_df

Call `preprocess_apple()` on `apple_df` and check if it was properly cleaned:

In [9]:
clean_apple_df = preprocess_apple()
clean_apple_df.head(10)

,transaction_date,merchant,category,amount,card
0,05/27/2023,khan shabu shabu and b,Food & Drink,96.49,Apple
1,05/25/2023,safeway,Groceries,24.22,Apple
2,05/23/2023,dino's grill,Food & Drink,39.26,Apple
3,05/23/2023,osaka marketplace,Groceries,9.30,Apple
4,05/19/2023,mcdonald's,Food & Drink,9.06,Apple
5,05/18/2023,safeway,Groceries,36.43,Apple
6,05/17/2023,best buy,Other,76.55,Apple
7,05/16/2023,habit fremont #93 q91,Food & Drink,33.42,Apple
8,05/12/2023,mcdonald's,Food & Drink,10.49,Apple
9,05/13/2023,cal alumni association,Other,340.00,Apple


All three DataFrames have been successfully cleaned; now to combining them all into one DataFrame.

### Table Concatenation

With all the preprocessing done, I combine all three DataFrames together to get my final `spending_df`:

In [18]:
# Conccat all three DataFrames together
spending_df = pd.concat([clean_boa_df, clean_chase_df, clean_apple_df])
spending_df = spending_df.reset_index(drop=True)
spending_df

,transaction_date,merchant,category,amount,card
0,05/09/2023,galpao gaucho cupertino,Food & Drink,175.44,BoA
1,04/28/2023,tst teaspoon - saratoga,Food & Drink,5.50,BoA
2,04/25/2023,chipotle 1031,Food & Drink,24.53,BoA
3,04/24/2023,chipotle 1031,Food & Drink,13.84,BoA
4,04/24/2023,dino's restaurant,Food & Drink,38.00,BoA
...,...,...,...,...,...
155,05/08/2023,hui lau shan,Food & Drink,13.51,Apple
156,05/07/2023,billy roys burger,Food & Drink,60.29,Apple
157,05/04/2023,marukai,Groceries,36.97,Apple
158,05/06/2023,yugo ramen,Food & Drink,41.17,Apple


All records have been successfully combined. Next is to export the file into a `.csv` so that I can create visualizations in Tableau.

In [19]:
spending_df.to_csv('spending.csv', index_label='index')